In [2]:
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.cleanup import *
import time
time1 = time.time()

my_auth = get_key('koray_data')

In [5]:
q = '/search/?experiments_in_set.biosample.biosource.individual.organism.name=chicken&experimentset_type=replicate&type=ExperimentSetReplicate'
set_to_release = [i['uuid'] for i in ff_utils.search_metadata(q, my_auth)]
        
                
def fetch_wfr_associated(wfr_uuid, my_key):
    """Given wfr_uuid, find associated output files and qcs"""
    wfr_as_list = []
    wfr_info = ff_utils.get_metadata(wfr_uuid, my_key)
    wfr_as_list.append(wfr_info['uuid'])
    if wfr_info.get('output_files'):
        for o in wfr_info['output_files']:
                if o.get('value'):
                    wfr_as_list.append(o['value']['uuid'])
                if o.get('value_qc'):
                    wfr_as_list.append(o['value_qc']) # this is a @id
    if wfr_info.get('output_quality_metrics'):
        for qc in wfr_info['output_quality_metrics']:
            if qc.get('value'):
                wfr_as_list.append(qc['value']['uuid'])
    return list(set(wfr_as_list))

search_with = 'hi-c-processing-pairs'
print(len(set_to_release))
for ix in set_to_release:
    my_list = []
    store = {}
    item_uuids = []
    store, uuids = ff_utils.expand_es_metadata([ix], my_auth, store_frame='object',add_pc_wfr=True, ignore_field = ['experiment_relation', 'biosample_relation', 'references'])
    time2 = time.time()
    print(len(uuids), round((time2-time1), 1), 'sec for collection')
    
    for wf in set([i['display_title'].split(' run')[0] for i in store.get('workflow_run_awsem')]):
        if search_with in wf:
            my_list = [i for i in store['workflow_run_awsem'] if i['display_title'].startswith(search_with)]
            break
    print(len(my_list))
    
    if my_list:
        del_list = fetch_wfr_associated(my_list[0]['uuid'], my_auth)
        print(len(del_list))
        
        for ele in del_list:
            # ff_utils.patch_metadata({'status': "deleted"}, obj_id = ele, key=my_auth)
            pass
        print('deleted items')
    print()


40
111 675.5 sec for collection
0

110 680.1 sec for collection
0

110 685.3 sec for collection
0

110 691.2 sec for collection
0

110 697.2 sec for collection
0

109 703.3 sec for collection
0

109 711.2 sec for collection
0

112 714.0 sec for collection
0

111 716.5 sec for collection
0

111 719.9 sec for collection
0

111 722.2 sec for collection
0

111 725.3 sec for collection
0

110 728.4 sec for collection
0

110 730.6 sec for collection
0

506 735.7 sec for collection
0

545 740.3 sec for collection
0

535 745.5 sec for collection
0



KeyboardInterrupt: 

In [8]:
q1 = '/search/?completed_processes=HiC_Pipeline_0.2.6&experiments_in_set.biosample.biosource.individual.organism.name=chicken&experimentset_type=replicate&type=ExperimentSetReplicate'
all_res = ff_utils.search_metadata(q1, my_auth)
   
print(len(all_res))

20


In [9]:
for a_set in all_res:
    ff_utils.patch_metadata({}, a_set['accession'], my_auth, add_on = 'delete_fields=processed_files,other_processed_files,completed_processes')